In [1]:
%cd ..

/tf/notebooks


In [2]:
import json
import re
import random
from tqdm import tqdm_notebook as tqdm

In [3]:
with open("data/fddb_data_v4_withimg.json", encoding='utf-8') as f:
    fddb = json.load(f)

In [4]:
with open("data/recipe-ingredient-to-fooddb.json") as f:
    all_ings_orig = json.load(f)

In [5]:
from extract_ingredients.util import normalize_out_ingredient


In [6]:
custom_amounts = {("Öl", "Olivenöl"): ["UNK (20 ml)", "etwas (20 ml)", "Esslöffel (10 ml)"], # 2EL = 20ml
                  ("Salz",): ["UNK (2 g)"],
                 }

custom_amounts = {ks: v for k, v in custom_amounts.items() for ks in k}
custom_amounts

{'Olivenöl': ['UNK (20 ml)', 'etwas (20 ml)', 'Esslöffel (10 ml)'],
 'Salz': ['UNK (2 g)'],
 'Öl': ['UNK (20 ml)', 'etwas (20 ml)', 'Esslöffel (10 ml)']}

In [7]:
def normalize_amount_unit(unit):
    if re.match(r"^mittelgroße[sre]?$", unit):
        return "mittelgroß"
    if unit == "Dose (Abtropfgewicht)":
        return "Dose"
    return unit

In [8]:
from collections import defaultdict
fddb_entries = defaultdict(list)
rem = re.compile(r"^(\d+ )?(.*) \((\d+(?:,\d+)?) (g|ml)\)$")
def handle_fddb():
    for e in tqdm(fddb):
        for ning in normalize_out_ingredient(e["name"]):
            fddb_entries[ning].append(e)
        sanity = list(e["Standard Nährwerte"].keys())
        snmatchs = [re.match("^Nährwerte für 100 (\w+)$", s) for s in sanity]
        snmatchs = [x for x in snmatchs if x is not None]
        if len(snmatchs) == 1:
            unn = snmatchs[0].groups()[0]
            e["parsed"] = {"unit": unn, "for_100": e["Standard Nährwerte"]["Nährwerte für 100 {}".format(unn)]}

        else:
            print("SANITY ERROR:", e["Id"], sanity, unit, snmatch)
            continue
        amts = e["parsed"]["amounts"] = []
        spec_keys = list(e["Spezifische Nährwerte"].keys())
        if e["name"] in custom_amounts:
            spec_keys += custom_amounts[e["name"]]
        for k in spec_keys:
            if k == "100 g (100 g)" or k == "100 g (100 ml)" or k == "100 ml (100 ml)":
                continue
            txtamount, txt, amount, unit = rem.match(k).groups()
            amount = float(amount.replace(",", "."))
            txtamount = float(txtamount.replace(",", ".")) if txtamount else 1.0
            txt = normalize_amount_unit(txt)
            amts.append((txt, k, amount/txtamount))

            if unn != unit:
                raise Exception("noooo " + unn + " " + unit, e["name"])
            #print(txt, amount/txtamount, unit)
handle_fddb()

In [9]:
# get recipe meta
with open("data/recipes/processed_data.json", encoding="utf-8") as f:
    recipes = json.load(f)

In [10]:
# get amount map from parse-amounts
with open("data/recipes/parsed_amounts.json") as f:
    parsed_amounts = json.load(f)

In [11]:
# only recipes with pictures
recipes = [recipe for recipe in recipes if len(recipe['picture_files']) > 0]

In [12]:
from operator import itemgetter
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from IPython.display import HTML
from html import escape
# ugly af code
debug=False
    
def match_amts(*, ing_matches, have_count, have_unit):
    possible_entries = {entry["Id"]: (ing_match[1], entry) for ing_match in ing_matches for entry in fddb_entries[ing_match[0]]}
    possible_entries = sorted(list(possible_entries.values()), key=lambda e: -e[0])
    have_unit = [normalize_amount_unit(unit) for unit in have_unit]
    for accuracy, e in possible_entries:
        if "parsed" not in e:
            continue
        debug and print("considering", e["name"], e["Id"])
        want_unit = e["parsed"]["unit"]

        if want_unit in have_unit:
            debug and print("got direct match!", have_count, have_unit, have_count, want_unit)
            return (e, accuracy, have_count, want_unit, have_count, want_unit)
        for amount_name, amount_source, to_norm in e["parsed"]["amounts"]:
            debug and print(e["name"], "amount", amount_source)
            if amount_name in have_unit:
                debug and print("got match unit!", amount_source)
                return (e, accuracy, have_count, have_unit[0], have_count*to_norm, want_unit)

def match_ingredient(ingredient):
    output = {"original": ingredient, "type": "ingredient" if "amount" in ingredient else "subtitle"}
    if output["type"] == "ingredient":
        amount_count, amount_type = parsed_amounts[ingredient["amount"]]

        if amount_count is None: # only happens when both amount_count and amount_type is None (else it is set to 1)
            #return cols + ["[ignored]"]
            output["matched"] = {"matched": False, "important": False, "message": "[unimportant]"}
            return output
            #amount_count = 1
            #amount_type = ["UNK"]
        if amount_type is None:
            amount_type = ["Stück", "mittelgroß", "UNK"]
        elif amount_type in ["dicke", "dicker"]:
            amount_type = [amount_type, "große", "großer"]
        else:
            amount_type = [amount_type]
        #print("[" + str(amount_count) + "*" + (amount_type)+ "] of ", end="")
        _ing_matches = all_ings_orig[ingredient["ingredient"]]
        # all matches better than 84% or the first one if it's worse
        ing_matches = [e for e in _ing_matches if e[1] > 0.84]
        if len(ing_matches) == 0:
            # not so great match but eh
            ing_matches = [_ing_matches[0]]


        debug and print("matches", ing_matches)
        debug and print("finding unit ", amount_type)
        match = match_amts(ing_matches=ing_matches, have_count=amount_count, have_unit=amount_type)

        if not match:
            message = "[no amount for match for [{}] on [{}]]".format("|".join(amount_type), "|".join(t for t, _ in ing_matches))
            # important if calorie dense (>100kcal/100g)
            important = fddb_entries[ing_matches[0][0]][0]["parsed"]["for_100"]["Kalorien"]["Menge"] > 100
            if not important:
                message = "[unimportant]" + message
            output["matched"] = {"matched": False, "important": important, "message": message}
        else:
            fddb_entry, accuracy, count_weird, unit_weird, count_norm, unit_norm = match
            output["matched"] = {
                "matched": True,
                "match_accuracy": accuracy,
                "weird": {"count": count_weird, "unit": unit_weird},
                "normal": {"count": count_norm, "unit": unit_norm},
                "name": fddb_entry["name"],
                "id": fddb_entry["Id"],
                "multiplier": count_norm / 100,
                "nutritional_values": fddb_entry["parsed"]["for_100"] #parse_nutritional_values(values)
            }
    return output

def disp_ing(ing):
    if ing["type"] == "ingredient":
        cols = [ing["original"]["amount"], ing["original"]["ingredient"], "="]
        p = ing["matched"]
        if not p["matched"]:
            return cols + [p["message"]]

        weird = p["weird"]
        norm = p["normal"]
        if weird["unit"] == norm["unit"]:
            unittxt = "{}{}".format(norm["count"], norm["unit"])
        else:
            unittxt = "{} {} = {}{}".format(weird["count"], weird["unit"], norm["count"], norm["unit"])
        values = p["nutritional_values"]
        value_kcal_per_100 = values["Kalorien"]["Menge"]
        value_kcal = value_kcal_per_100 * p["multiplier"]
        cols.append("({:.0f}%) [{}] of {} = {:.0f} kcal".format(p["match_accuracy"] * 100, unittxt, p["name"], value_kcal))
        return cols
    elif ing["type"] == "subtitle":
        return ["->>" + ing["original"]["subtitle"]]


In [13]:
def sum_nutritional_values(ings):
    out_vals = defaultdict(lambda: {"Menge": 0, "Einheit": None})
    for ing in ings:
        if ing["type"] != "ingredient":
            continue
        if not ing["matched"]["matched"]:
            if ing["matched"]["important"]:
                return None
            else:
                continue
        nutritional_values = ing["matched"]["nutritional_values"]
        multiplier = ing["matched"]["multiplier"]
        for k, v in nutritional_values.items():
            o = out_vals[k]
            o["Menge"] += v["Menge"] * multiplier
            bef_unit = o["Einheit"]
            if bef_unit is None:
                o["Einheit"] = v["Einheit"]
            if o["Einheit"] != v["Einheit"]:
                print(o, v)
                raise Exception("Unit mismatch")
    return out_vals

def match_recipe(recipe):
    ings = [match_ingredient(ing) for ing in recipe["ingredients"]]
    summed_nut = sum_nutritional_values(ings)
    if summed_nut is not None:
        nut = {
        "per_portion": {k: {**v, "Menge": v["Menge"] / recipe["portions"]} for k, v in summed_nut.items()},
        "per_recipe": summed_nut
    }
    else:
        nut = None
    return {**recipe, "ingredients": ings, "nutritional_values": nut}

In [14]:
match_recipe(recipes[104])["nutritional_values"]

{'per_portion': {'Ballaststoffe': {'Einheit': 'g', 'Menge': 1.065},
  'Brennwert': {'Einheit': 'kJ', 'Menge': 1558.5500000000002},
  'Broteinheiten': {'Einheit': '', 'Menge': 1.76},
  'Cholesterin': {'Einheit': 'mg', 'Menge': 40.8},
  'Fett': {'Einheit': 'g', 'Menge': 28.985},
  'Kalorien': {'Einheit': 'kcal', 'Menge': 372.53000000000003},
  'Kohlenhydrate': {'Einheit': 'g', 'Menge': 21.15},
  'Protein': {'Einheit': 'g', 'Menge': 7.055},
  'Wassergehalt': {'Einheit': '%', 'Menge': 19.71},
  'davon Zucker': {'Einheit': 'g', 'Menge': 4.99}},
 'per_recipe': defaultdict(<function __main__.sum_nutritional_values.<locals>.<lambda>()>,
             {'Ballaststoffe': {'Einheit': 'g', 'Menge': 2.13},
              'Brennwert': {'Einheit': 'kJ', 'Menge': 3117.1000000000004},
              'Broteinheiten': {'Einheit': '', 'Menge': 3.52},
              'Cholesterin': {'Einheit': 'mg', 'Menge': 81.6},
              'Fett': {'Einheit': 'g', 'Menge': 57.97},
              'Kalorien': {'Einheit': 'kca

In [15]:
def display_recipe(recipe):
    display(HTML("<h2>{}</h2>".format(escape(recipe["title"]))))
    ings = match_recipe(recipe)["ingredients"]
    #print(ings)

    ings_df = pd.DataFrame([disp_ing(ing) for ing in ings], columns=["inp amount", "inp ing", "", "paredd"])
    display(HTML(ings_df.to_html()))
                                      
for recipe in random.sample(recipes, 20):
    display_recipe(recipe)

,inp amount,inp ing,,paredd
0,500 g,"Quinoa, schwarz und weiß gemischt",=,(100%) [500g] of Quinoa = 1830 kcal
1,,Salzwasser,=,[no amount for match for [Stück|mittelgroß|UNK] on [Algensalat]]
2,2,"Zwiebel(n), rot",=,(88%) [2 Stück = 400.0g] of Gemüsezwiebel = 124 kcal
3,2 große,"Paprikaschote(n), rot",=,[no amount for match for [große] on [Paprikasülze|Paprikastreifen|Paprikagulasch|Gefüllte Paprikaschote|Paprikapastete|Paprikagemüse|paprikaschoten|Paprikasticks|Paprikareis|Paprika Edelsüss|Paprikapulver|Paprikaschoten gegrillt|Paprikasalat|Paprikamark|Paprika-Mandel-Dip|Eingelegte Paprika|Chrunchips Paprika|Paprika-Bruscetta|Pfeffer-Braten|Chilischote|Paprikacreme|2 Paprikaschoten|Pfefferwildtopf|Paprikasoße|Paprika-Lyoner|Kartoffelringe Paprika|Gefüllte Paprika|Paprikasuppe|Paprika-Olive Aufstrich|Pfefferonen]]
4,150 g,Rosinen,=,(100%) [150g] of Rosinen = 471 kcal
5,2 Stängel,Koriandergrün,=,[no amount for match for [Stängel] on [Koriander-Grün-Paste|Mandarinen-Orangen]]
6,1 TL,Kreuzkümmel,=,[no amount for match for [Teelöffel] on [Schwarzkümmel|Kümmelpärle|Kümmel|Kümmelstange|Kümmelfleischmagen|Kümmelbrot|Kümmelbraten|Kräuterschinken|Lammleber|Kreuzkümmel (Cumin)|Kräuterbutter|Knoblauchpüree|Schwarzkümmelöl|Leberknödel|Fenchel-Anis-Kümmel|Kümmelbraten geschniten|Krautwickel|Pfefferschinken|Mehlknödel|Kräuterbutterfilet|Butterschinken|Gewürzschinken|Rote Grütze|Dreikornbüsch|Kräutersoße|Kräuterpastete|Körnersemmel|Knoblauchbutter|Roggensemmel|schwarzkümmelöl]]
7,1 TL,Currypulver,=,"[no amount for match for [Teelöffel] on [Currypulver|Curry Powder|Currygewürz, Pulver|Curry-Knacker|Cremiges Curry|Curry-Frikadellen|Curry Paste|Paprikapulver|Curry-Wurst|Currysoße|Rote Currypaste|Curryreis|Kokos Curry|Currysauce|Curry Gewürzzubereitung]]"
8,,Chilipulver,=,"(89%) [1 Stück = 5.0g] of Chilischote, frisch = 2 kcal"
9,80 ml,Apfelessig,=,(100%) [80ml] of Apfelessig = 14 kcal


,inp amount,inp ing,,paredd
0,500 g,Hähnchenleber(n),=,(96%) [500g] of Hähnchenleber = 680 kcal
1,500 g,"Herz(en), (Hähnchen-)",=,(82%) [500g] of nussherz = 1525 kcal
2,1 Glas,Okraschoten,=,[unimportant][no amount for match for [Glas] on [Okra|paprikaschoten|okra]]
3,2 Dose/n,Kichererbsen,=,(100%) [2 Dose = 530.0g] of Kichererbsen = 429 kcal
4,200 ml,Kokosmilch,=,(100%) [200ml] of Kokosmilch = 358 kcal
5,2,Zwiebel(n),=,(88%) [2 Stück = 400.0g] of Gemüsezwiebel = 124 kcal
6,2 Zehe/n,Knoblauch,=,"(89%) [2 Zehe = 6.0g] of Knoblauch, frisch = 9 kcal"
7,1 Stück(e),Ingwerwurzel,=,(86%) [1 Stück = 4.0g] of Ingwer-Zitrone = 14 kcal
8,1 EL,Rapsöl,=,(100%) [1 Esslöffel = 10.0g] of Rapsöl = 83 kcal
9,2 Dose/n,"Tomate(n), gehackt",=,(90%) [2 Dose = 800.0g] of Tomatenstücke = 192 kcal


,inp amount,inp ing,,paredd
0,300 g,Mehl,=,(100%) [300g] of Mehl = 1044 kcal
1,½ TL,Backpulver,=,(100%) [0.5 Teelöffel = 1.5g] of Backpulver = 2 kcal
2,85 g,Zucker,=,(100%) [85g] of Zucker = 344 kcal
3,1 Pck.,Vanillezucker,=,(100%) [1 Packung = 8.0g] of Vanillezucker = 32 kcal
4,½ Pck.,Rumaroma,=,[unimportant][no amount for match for [Packung] on [Plusaroma]]
5,130 g,Butter,=,(100%) [130g] of Butter = 971 kcal
6,5,"Apfel, nicht zu sauer",=,"(100%) [5 Stück = 625.0g] of Apfel, roh = 338 kcal"
7,1 Glas,Aprikosenkonfitüre,=,(93%) [1 Glas = 450.0g] of Konfitüre Aprikose = 1130 kcal
8,->>Für die Streusel:,None,None,None
9,200 g,Mehl,=,(100%) [200g] of Mehl = 696 kcal


,inp amount,inp ing,,paredd
0,1 Scheibe/n,Brot(e) (Schwarz-),=,(96%) [1 Scheibe = 60.0g] of Brot = 172 kcal
1,1,Käse (Ziegen-),=,(90%) [1 Stück = 36.0g] of Butterkäse = 114 kcal
2,1 EL,Honig,=,(100%) [1 Esslöffel = 15.0g] of Honig = 49 kcal
3,1 EL,Linsen (Beluga-),=,[unimportant][no amount for match for [Esslöffel] on [Linsen|Feine Linsen|Linsen Aufstrich|Linsensprossen|Teller Linsen|Linsen - gegart|Schwarze Linsen|Berg Linsen|Tellerlinsen|Grüne Linsen|Tempo Linsen|Rote Linsen|Linsen dal|Bio Linsen|Linsen grün|Linsen braun|grüne Linsen|rote Linsen|Gelbe Linsen|Linsen Glück|Tempolinsen|linseneintopf|Linsen Liebe|Braune Linsen]]
4,n. B.,Petersilie,=,[unimportant]


,inp amount,inp ing,,paredd
0,16,"Riesengarnelen (King Prawns), geschält, entdarmt",=,"[unimportant][no amount for match for [Stück|mittelgroß|UNK] on [Riesengarnelen|Riesenfgarnelem|Riesengarnele|Riesengarnelenschwnze|Gourmet Riesengarnelen|riesengarnelen|Riesengarnelenschwänze|Tigergarnele|Atlantische Rotgarnelen|Riesengarnelen, Provencal|King Prawns Riesengarnelen|Argentinische Rotgarnelen|Eismeergarnelen]]"
1,250 g,Schweineschnitzel,=,(100%) [250g] of Schweineschnitzel = 288 kcal
2,600 g,"Pilze (Champignons, Austernpilze)",=,(100%) [600g] of Pilze = 954 kcal
3,10,Bohnen (Schlangenbohnen),=,(87%) [10 Stück = 26.0g] of Saure Bohnen = 88 kcal
4,5,"Chilischote(n), rot, lang",=,"(94%) [5 Stück = 25.0g] of Chilischote, frisch = 10 kcal"
5,½ EL,"Garnelenpaste, thailändische (Gapi)",=,[unimportant][no amount for match for [Esslöffel] on [Garnelen-Spieße|Garnelen|Garnelen-Pfanne|Pfannen Garnelen|Marinierte Garnelen|Thunfischfrikadelle|Garnelen Provencale|Gourmet Garnelen|Garnelenspieße|Fischfrikadelle]]
6,10,"Chilischote(n) (Vogelaugenchilies, 25 im Original, wer sich traut)",=,"(94%) [10 Stück = 50.0g] of Chilischote, frisch = 20 kcal"
7,5,Knoblauchzehe(n) (25 im Original),=,(93%) [5 Stück = 20.0g] of Knoblauchpüree = 28 kcal
8,1 kleine,"Zwiebel(n), wenn man nur 5 Knoblauchzehen nimmt",=,"[unimportant][no amount for match for [kleine] on [Zwiebeln|Zwiebel|Zwiebelringe|Zwiebelmett|Zwiebelsenf|Zwiebelschmalz|Rote Zwiebeln|Zwiebeln, gegart|Rstzwiebeln|Zwiebelfleisch|Gemüsezwiebel|Zwiebel Ringe|Zwiebel Schmelz|Zwiebelwürfel|Süsse Zwiebel|Zwiebeln gewürfelt|Zwiebeln, gewürfelt|Zwiebelringe, Zwiebel|Zwiebelflute|Rote Zwiebel|Jungzwiebel|Zwiebel-Sülze|Sielberzwiebeln|Zwiebel, getrocknet|Silberzwiebeln|silberzwiebeln|Zwiebelstange, Zwiebel|Zwiebel, frisch]]"
9,½,"Limette(n), Saft davon, bei Bedarf mehr",=,"(93%) [0.5 Stück = 32.5g] of Limette, frisch = 16 kcal"


...

,inp amount,inp ing,,paredd
0,->>Für den Teig:,None,None,None
1,300 g,Weizenmehl,=,(100%) [300g] of Weizenmehl = 1032 kcal
2,4 TL,Backpulver,=,(100%) [4 Teelöffel = 12.0g] of Backpulver = 12 kcal
3,5 EL,Öl (z. B. Rapsöl),=,(100%) [5 Esslöffel = 50.0ml] of Öl = 400 kcal
4,200 ml,Buttermilch,=,(100%) [200ml] of Buttermilch = 70 kcal
5,,Salz,=,(100%) [1 Stück = 2.0g] of Salz = 0 kcal
6,->>Für den Belag:,None,None,None
7,12,Zwiebel(n),=,(88%) [12 Stück = 2400.0g] of Gemüsezwiebel = 744 kcal
8,3 EL,Öl,=,(100%) [3 Esslöffel = 30.0ml] of Öl = 240 kcal
9,3 Becher,Frischkäse (Frischer Landrahm),=,(100%) [3 Becher = 525.0g] of Frischkäse = 840 kcal


,inp amount,inp ing,,paredd
0,1,Salatgurke(n),=,[unimportant][no amount for match for [Stück|mittelgroß|UNK] on [Salatgurke|gurkensalat|Jungsalat|Königssalat|Randensalat|Hirtensalat|Blattsalat|Kopfsalat|Feldsalat|Heringsalat|Lauchsalat|Pußtasalat|Salatveredler|Fertigsalat|SalatBouquet|Geflgelsalat|Pflücksalat|Salat|Salatwürze|Randen Salat|Bauernsalat|Heringssalat|Fleischpastete|Kalbskotelett|Salatmischung|Bohnensalat|Salatherzen|Selleriesalat|Fleischsalat|Bistrogurken]]
1,1 EL,Reisessig,=,[unimportant][no amount for match for [Esslöffel] on [Reisessig|Obstessig|Tafelessig|Bäuerliches|Laugenecke|Klarheit|Laugen|Steckrübe|Bunter Genuss|Elasten|Regenerate|Schäufele|Laugenbrezen|Laugenwuchtel|Estragonsenf|Laugenzöpfli|Glüser|Bärchen-Streich|Mairübchen]]
2,1 EL,Wasser,=,"[unimportant][no amount for match for [Esslöffel] on [Wasser|wasser|Mineralwasser|Wasser, still]]"
3,1 EL,Sojasauce,=,(95%) [1 Esslöffel = 10.0ml] of Soja Sauce = 6 kcal
4,1 TL,"Chilipulver, koreanisches (Gotchugaru)",=,(90%) [1 Teelöffel = 3.0g] of Backpulver = 3 kcal
5,1 TL,Salz,=,(100%) [1 Teelöffel = 7.5g] of Salz = 0 kcal
6,1 Zehe/n,Knoblauch,=,"(89%) [1 Zehe = 3.0g] of Knoblauch, frisch = 4 kcal"
7,1 TL,"Sesamöl, dunkles",=,(88%) [1 Teelöffel = 5.0ml] of Trüffelöl = 41 kcal
8,1 TL,"Sesam, geröstet",=,"[no amount for match for [Teelöffel] on [Sesam|Sesamring|Wass Sesam|Sesamstange|Sesammus|Sesam, geschält|Sesamstangen|schwarzer Sesam]]"


,inp amount,inp ing,,paredd
0,250 g,Spaghettini,=,(100%) [250g] of Spaghettini = 880 kcal
1,1 Bund,Petersilie,=,"(88%) [1 Bund = 35.0g] of Petersilie, frisch = 18 kcal"
2,100 g,Parmesan,=,(89%) [100g] of parmesan = 402 kcal
3,250 g,Pilze,=,(100%) [250g] of Pilze = 398 kcal
4,½ Knolle/n,Knoblauch,=,"[no amount for match for [Knolle] on [Knoblauch|Knoblauchgurken|Knoblauchpüree|Knoblauchschmalz|Knoblauchecken|Knoblauchsauce|Knoblauchcreme|Knoblauchbraten|Knoblauch, getrocknet|Knoblauchbutter|Knoblauchsoße|Knoblauch Braten|Knoblauch sauce|Knoblauch sauce|Knoblauch-Baguette|Knoblauchkick|Knoblauchsalami|Knoblauch-Creme|Knoblauchsuppe|Knoblauch Senf|Knoblauchbeißer|Knoblauch, frisch|Knoblauchbaguette|Knoblauch Dressing|Knoblauchecken, Knoblauch|Süß-Sauer Knoblauch|Öl, Knoblauch|Knoblauch geröstet|Knoblauchsalz|Knoblauch natur]]"
5,50 g,Butter,=,(100%) [50g] of Butter = 374 kcal
6,150 ml,Sahne,=,(100%) [150ml] of Sahne = 440 kcal
7,1 TL,Speisestärke,=,"[no amount for match for [Teelöffel] on [Speisestärke|Feine Speisestärke|Speisetopfen|feine Speisestärke|Speisetopfen mager|Speisequark|Kraftklotz|Kloßmasse|Quarkspeise|Speise Quark|gemüseaufstrich|Reistörtchen|Speisetopfen, mager|Kraftfleisch Aufschnitt|Rohscheiben]]"
8,1 TL,"Gemüsebrühe, gekörnte",=,"[unimportant][no amount for match for [Teelöffel] on [Gemüsebrühe|Gemüse-Hefebrühe|Klare Gemüsebrühe|klare Gemüsebrühe|Rinderbrühe|Gemüseeintopf|Gemüsebrei|Hühnerbrühe|Gemüsewähe|Bio Gemüsebrühe|Gemüsewürzpaste|Gemüse-Brühe|Gemüsebrühe, Würfel|Gemüserösti|Gemüsebouletten|Gemüsesülze|Gemüsebrühe Bio|Gemüsemischung|Gemüsebrühe - rein pflanzlich|Gemüseomelett|Rahmgemüse|Gemüsesuppe|Gemüsekorb|Gemüsebouillon|cleane Gemüsebrühe|Gemüsebrühe, Gemüse|Gemüse Eintopf|Gemüse Eintopf|Gemüseaufstrich|Gemüsescheibe]]"
9,1,Zitrone(n),=,(93%) [1 Stück = 70.0g] of Zitronenrolle = 193 kcal


,inp amount,inp ing,,paredd
0,150 g,Kekse (Oreos),=,(100%) [150g] of Kekse = 100 kcal
1,1 Glas,"Sauerkirschen, entsteint, Abtropfgewicht 370 g",=,(96%) [1 Glas = 185.0g] of Süßkirschen = 74 kcal
2,"7 EL, gehäuft",Puderzucker,=,"[no amount for match for [Esslöffel, gehäuft] on [Puderzucker|Backzucker|Rohrzucker|Gelierzucker|Vanillezucker|Palmzucker|Rohrrohzucker|puderzucker|Milchzucker|Feinzucker|Roh-Rohrzucker|Gelier-Fruchtzucker|Rohrzucker grob|Hagelzucker|Vanillinzucker|Fruchtzucker|Traubenzucker|Zitronenzucker|Birkenzucker Kristallin|Würfelzucker|Rohrohrzucker|Vollrohrzucker|Kandiszucker braun|Zucker-Streusel|Gelvite Gelierzucker|Zuckerbrezel|Raffinade Zucker|Brauner Rohrzucker|Brauner Rohrzucker|Kokosblütenzucker]]"
3,2 EL,Speisestärke,=,(92%) [2 Esslöffel = 18.0g] of Feine Speisestärke = 64 kcal
4,etwas,Wasser,=,[unimportant]
5,5 EL,"Kirschwasser, optional",=,"[unimportant][no amount for match for [Esslöffel] on [Kirschsaft|Sauerkirsche|Kirschwein|Sauerkirschen|Kirschgrütze|Kirschwasser-Eier|Kirschenwähe|Sauer kirschen|Herzkirsche|Kirschen|Schwarzkirsche|Kirsch Grütze|Rosenwasser|Likörkirschen|Sauer- Kirschen|Sauerkirschen getrocknet|Kirschmarmelade|Sauerkirschen, entsteint|Kirschstreusel|Belegkirschen|Saure Kirschen|Süßkirschen]]"
6,200 ml,"Kirschsaft, evtl. weniger",=,(100%) [200ml] of Kirschsaft = 100 kcal
7,250 g,Mascarpone,=,(100%) [250g] of Mascarpone = 962 kcal
8,250 g,Magerquark,=,(100%) [250g] of Magerquark = 170 kcal
9,250 ml,Sahne,=,(100%) [250ml] of Sahne = 732 kcal


,inp amount,inp ing,,paredd
0,100 g,Feldsalat,=,(100%) [100g] of Feldsalat = 21 kcal
1,8,Cocktailtomaten,=,[unimportant][no amount for match for [Stück|mittelgroß|UNK] on [Cocktailgürkchen|Cocktail-Tomaten|Cocktail-Garnelen|Cocktail-Rispentomaten|Cocktailcrevetten|Cocktailsauce]]
2,2 EL,Olivenöl,=,(100%) [2 Esslöffel = 20.0ml] of Olivenöl = 164 kcal
3,1 EL,Balsamico,=,(100%) [1 Esslöffel = 15.0ml] of Balsamico = 22 kcal
4,1,Knoblauchzehe(n),=,(93%) [1 Stück = 4.0g] of Knoblauchpüree = 6 kcal
5,,Salz und Pfeffer,=,(100%) [1 Stück = 2.0g] of Salz = 0 kcal
6,100 g,Feta-Käse,=,(88%) [100g] of Creme-Käsle = 228 kcal
7,1,Ei(er),=,"(100%) [1 Stück = 55.0g] of Ei, vom Huhn = 75 kcal"
8,,Sesam,=,"[no amount for match for [Stück|mittelgroß|UNK] on [Sesam|Sesamring|Wass Sesam|Sesamstange|Sesammus|Sesam, geschält|Sesamstangen|schwarzer Sesam]]"
9,,"Olivenöl, zum Anbraten",=,(100%) [1 Stück = 20.0ml] of Olivenöl = 164 kcal


In [16]:
unm_ings = []
for recipe in random.sample(recipes, 200):
    recipe = match_recipe(recipe)
    for ing in recipe["ingredients"]:
        if ing["type"] == "ingredient" and not ing["matched"]["matched"]:
            if ing["matched"]["important"]:
                unm_ings.append(ing)

with pd.option_context('display.max_colwidth', 80):
    ings_df = pd.DataFrame([disp_ing(ing) for ing in unm_ings], columns=["inp amount", "inp ing", "", "paredd"])
    display(HTML(ings_df.to_html()))

,inp amount,inp ing,,paredd
0,1 TL,"Paprikapulver, edelsüß, edelsüß",=,[no amount for match for [Teelöffel] on [Paprikapulver|Paprikagulasch|Paprik...
1,3,Lorbeerblätter,=,"[no amount for match for [Stück|mittelgroß|UNK] on [Lorbeer, getrocknet]]"
2,2 Stange/n,Lauch,=,"[no amount for match for [Stange] on [Lauchschinken|Lauch, Porree]]"
3,4 EL,Sahne,=,[no amount for match for [Esslöffel] on [Sahne|Saure Sahne|Sahnestand|saure ...
4,1 TL,"Paprikapulver, (scharf)",=,[no amount for match for [Teelöffel] on [Paprikapulver|Paprikagulasch|Paprik...
5,1 EL,Margarine,=,"[no amount for match for [Esslöffel] on [Margarine|Margarine, leicht|Halbfet..."
6,½ TL,"Orange(n) - Schale, abgeriebene",=,[no amount for match for [Teelöffel] on [Orangen Schale gerieben|Orangensch...
7,,"Margarine, für die Form",=,"[no amount for match for [Stück|mittelgroß|UNK] on [Margarine|Margarine, lei..."
8,,Paniermehl,=,[no amount for match for [Stück|mittelgroß|UNK] on [Paniermehl|paniermehl|Pa...
9,2,"Vanilleschote(n), längs halbiert",=,[no amount for match for [Stück|mittelgroß|UNK] on [Vanilleschote|Zuckerscho...


In [17]:
with open("data/recipes/recipes_matched.jsonl", "w", buffering=2**20) as f:
    debug=False
    for recipe in tqdm(recipes):
        json.dump(match_recipe(recipe), f)
        f.write("\n")

In [18]:
fddb_entries["Ei, vom Huhn"]

[{'Bewertungen': '49',
  'Bilder': [{'title': 'Ei, vom Huhn | Hochgeladen von: JuliFisch',
    'url': 'https://fddb.info/static/db/982/GD7HBWHAHJ5J0W725JAPZCE6_278x208.jpg'},
   {'title': 'Hühnereier | Hochgeladen von: Thomas Bohlmann',
    'url': 'https://fddb.info/static/db/108/foto-mehrere-huehnereier_48x48.jpg'},
   {'title': 'Ei, vom Huhn von lvennewa809 | Hochgeladen von: lvennewa809',
    'url': 'https://fddb.info/static/db/308/UKFJES3707LWXCRZCQY9OQP7_48x48.jpg'}],
  'Hersteller': 'Naturprodukt',
  'Id': 'https://fddb.info/db/de/lebensmittel/naturprodukt_ei_vom_huhn/index.html',
  'Lebensmittelgruppe': 'Eier',
  'Quelle': 'Schätzung, Intern, USDA Nutrient Database. Die Produktdaten wurden am 04.03.2009 von einem Fddb Nutzer erhoben. Hinweise zu den Produktdaten. Aktualisiert: 22.12.2016.   ',
  'Spezifische Nährwerte': {'1 Stück XL (73 g)': {'Brennwert': {'Einheit': 'kJ',
     'Menge': '418,'},
    'Kalorien': {'Einheit': 'kcal', 'Menge': '100'}},
   '1 großes (63 g)': {'Brennw

In [19]:
normalize_out_ingredient("Ei, vom Huhn")

['Ei, vom Huhn', 'Ei', 'Eier']

In [20]:
match_ingredient({"amount": "1 Pck.", "ingredient": "Backpulver"})

{'matched': {'id': 'https://fddb.info/db/de/lebensmittel/ruf_backpulver/index.html',
  'match_accuracy': 1.0000001192092896,
  'matched': True,
  'multiplier': 0.15,
  'name': 'Backpulver',
  'normal': {'count': 15.0, 'unit': 'g'},
  'nutritional_values': {'Brennwert': {'Einheit': 'kJ', 'Menge': 366},
   'Broteinheiten': {'Einheit': '', 'Menge': 1.8},
   'Fett': {'Einheit': 'g', 'Menge': 0},
   'Kalorien': {'Einheit': 'kcal', 'Menge': 87},
   'Kohlenhydrate': {'Einheit': 'g', 'Menge': 21.4},
   'Protein': {'Einheit': 'g', 'Menge': 0.1},
   'Wassergehalt': {'Einheit': '%', 'Menge': 10},
   'davon Zucker': {'Einheit': 'g', 'Menge': 0}},
  'weird': {'count': 1, 'unit': 'Packung'}},
 'original': {'amount': '1 Pck.', 'ingredient': 'Backpulver'},
 'type': 'ingredient'}

In [21]:
z = fddb_entries["Backpulver"][2]["parsed"]["for_100"]["Kalorien"]["Menge"]
float(z.replace(",", "."))

AttributeError: 'int' object has no attribute 'replace'

In [ ]:
match_ingredient({"amount": "1", "ingredient": "Schalotte(n)"})

In [ ]:
match_ingredient({"amount": "1 Glas", "ingredient": "Schattenmorellen"})

In [ ]:
match_ingredient({"amount": "3 m.-große", "ingredient": "Ei(er)"})

In [ ]:
match_ingredient({"amount": "", "ingredient": "Olivenöl"})

In [ ]:
match_ingredient({"amount": "300 EL", "ingredient": "Öl"})

In [ ]:
match_ingredient({"amount": "1", "ingredient": "Karotte(n)"})

In [ ]:
all_ings_orig["Karotte(n)"]